Notebook voor het genereren van data voor het trainen van Tesseract-OCR. De package 'trdg' is hiervoor zelf aangepast om data te genereren in kenteken format. 

In [1]:
%load_ext pycodestyle_magic
%flake8_on --max_line_length 119
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import os
import pytesseract
import re
import random
import string
import PIL
from random import randrange
from trdg.generators import (
    GeneratorFromDict,
    GeneratorFromRandom,
    GeneratorFromStrings,
    GeneratorFromWikipedia,
)

# Vervang met jou eigen path
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [3]:
path = r"../DataSet/TesseractTraining/"

In [4]:
def create_strings_randomly(count=10, let=True, num=True, lang='eng'):
    """
        Create all strings by randomly sampling from a pool of characters.
    """

    pool = ""
    if let:
        if lang == "cn":
            pool += "".join(
                [chr(i) for i in range(19968, 40908)]
            )  # Unicode range of CHK characters
        else:
            pool += string.ascii_letters
    if num:
        pool += "0123456789"

    strings = []
    for _ in range(0, count):
        current_string = "-XXX-"
        front = randrange(1, 4)*"X"
        back = randrange(1, 4)*"X"
        current_string = front+current_string+back
        current_string = list(current_string)
        i = -1
        for letter in current_string:
            i += 1
            if letter == "-":
                continue
            current_string[i] = random.choice(pool).upper()
        current_string = "".join(current_string)
        strings.append(current_string)
    return strings

In [10]:
def create_new_img(string, skewing_angle=0, blur=0):
    # Load image
    img = PIL.Image.open(path+r"plate_template/plate1.jpg")
    draw = PIL.ImageDraw.Draw(img)

    # Set font size depending on length of string
    if len(string) >= 10:
        font = PIL.ImageFont.truetype("GIL_____.TTF", 90)
    else:
        font = PIL.ImageFont.truetype("GIL_____.TTF", 100)
    width, height = img.size

    # Draw the text
    draw.text(((width/2)+20, (height/2)-10), string, (0, 0, 0), font=font, anchor="mm")

    # Random angle
    random_angle = random.randint(0 - skewing_angle, skewing_angle)
    img = img.rotate(random_angle)

    # Random blurring
    gaussian_filter = PIL.ImageFilter.GaussianBlur(random.randint(0, blur))
    img = img.filter(gaussian_filter)

    img.save(path+r"plate_model_6-ground-truth/"+string+".png")

In [11]:
def plate_generator(count=10, skewing_angle=0, blur=0):
    strings = create_strings_randomly(count)
    print(len(strings))

    counter = 0
    for string in strings:
        counter += 1
        if counter % 500 == 0:
            print("Counter is at", counter)

        create_new_img(string, skewing_angle, blur)
        text = re.sub(r'[^\w]', ' ', string)
        with open(path+r"plate_model_6-ground-truth/"+string+".gt.txt", "w") as text_file:
            text_file.write(text)

In [12]:
plate_generator(count=180000, skewing_angle=5, blur=5)

180000
Counter is at 500
Counter is at 1000
Counter is at 1500
Counter is at 2000
Counter is at 2500
Counter is at 3000
Counter is at 3500
Counter is at 4000
Counter is at 4500
Counter is at 5000
Counter is at 5500
Counter is at 6000
Counter is at 6500
Counter is at 7000
Counter is at 7500
Counter is at 8000
Counter is at 8500
Counter is at 9000
Counter is at 9500
Counter is at 10000
Counter is at 10500
Counter is at 11000
Counter is at 11500
Counter is at 12000
Counter is at 12500
Counter is at 13000
Counter is at 13500
Counter is at 14000
Counter is at 14500
Counter is at 15000
Counter is at 15500
Counter is at 16000
Counter is at 16500
Counter is at 17000
Counter is at 17500
Counter is at 18000
Counter is at 18500
Counter is at 19000
Counter is at 19500
Counter is at 20000
Counter is at 20500
Counter is at 21000
Counter is at 21500
Counter is at 22000
Counter is at 22500
Counter is at 23000
Counter is at 23500
Counter is at 24000
Counter is at 24500
Counter is at 25000
Counter is at

In [16]:
500%400 == 0

False

1:4: E228 missing whitespace around modulo operator
